In [1]:
# Example notebook to create dataframes from pbn files. Compatible with jupyter and vscode.
# Minimal documentation provided. Assumes user is familiar with github, jupyter/vscode notebook and python.

# author: Robert Salita (research@aipolice.org)
# 1. read a pbn file (local file).
# 2. create a df of deals, par, double dummy, single dummy probabilities, expected values, best contract (max expected value contract).

# install:
# 1. git clone https://github.com/BSalita/Calculate_PBN_Results
# 2. pip install -U -r requirements.txt

# requirements (specific to this project):
# pandas
# endplay
# pathlib

In [2]:

import pathlib
import pandas as pd
from collections import defaultdict

from endplay.parsers import pbn
from endplay.types import Deal, Contract, Denom, Player, Penalty
from endplay.dds import par, calc_all_tables
from endplay.dealer import generate_deals


In [3]:
# configurations
direction_order = [0,2,1,3] # NSEW order
suit_order = [3,2,1,0,4] # SHDCN order?
pbn_filename = 'DDS_Camrose24_1- BENCAM22 v WBridge5.pbn' # local filename
sd_productions = 100 # number of random deals to generate for calculating single dummy probabilities. Use smaller number for testing.

In [4]:
pd.options.display.max_columns = 0
#pd.options.display.min_rows = 20

In [5]:
# read local pbn file
pbn_file = pathlib.Path(pbn_filename)
with open(pbn_file, 'r') as f:
    boards = pbn.load(f)
len(boards), vars(boards[0])

(320,
 {'deal': Deal('N:T5.982.874.AQ632 K43.73.KQ5.KJT54 AJ9.AQT6.JT62.98 Q8762.KJ54.A93.7'),
  'auction': [PenaltyBid(penalty=<Penalty.passed: 1>, alertable=False, announcement=None),
   ContractBid(denom=<Denom.clubs: 3>, level=1, alertable=False, announcement=None),
   PenaltyBid(penalty=<Penalty.doubled: 2>, alertable=False, announcement=None),
   ContractBid(denom=<Denom.spades: 0>, level=1, alertable=False, announcement=None),
   PenaltyBid(penalty=<Penalty.passed: 1>, alertable=False, announcement=None),
   ContractBid(denom=<Denom.nt: 4>, level=1, alertable=False, announcement=None),
   PenaltyBid(penalty=<Penalty.passed: 1>, alertable=False, announcement=None),
   ContractBid(denom=<Denom.hearts: 1>, level=2, alertable=False, announcement=None),
   PenaltyBid(penalty=<Penalty.passed: 1>, alertable=False, announcement=None),
   ContractBid(denom=<Denom.spades: 0>, level=2, alertable=False, announcement=None),
   PenaltyBid(penalty=<Penalty.passed: 1>, alertable=False, announce

In [6]:
# create df from boards
df = pd.DataFrame([vars(b) for b in boards])
for col in df.columns:
    print(col, df[col].dtype)
    if df[col].dtype == 'object':
        if isinstance(df[col][0], dict):
            df = pd.concat([df,pd.DataFrame.from_records(df[col])],axis='columns')
df

deal object
auction object
play object
board_num int64
_dealer int64
_vul int64
_contract object
claimed bool
info object


,deal,auction,play,board_num,_dealer,_vul,_contract,claimed,info,Event,Site,Date,West,North,East,South,Scoring,BCFlags,Room,Score
0,N:T5.982.874.AQ632 K43.73.KQ5.KJT54 AJ9.AQT6.J...,"[P, 1♣, X, 1♠, P, 1NT, P, 2♥, P, 2♠, P, P, P]","[♦8, ♦5, ♦T, ♦A, ♣7, ♣A, ♣4, ♣8, ♠5, ♠3, ♠9, ♠...",1,0,0,2♠W+1,False,{'Event': '<u>Camrose 2024: BEN vs WBridge5</u...,<u>Camrose 2024: BEN vs WBridge5</u>,,2023.12.15,WBridge5,BENCAM22,WBridge5,BENCAM22,IMP,df,Open,EW 140
1,N:T5.982.874.AQ632 K43.73.KQ5.KJT54 AJ9.AQT6.J...,"[P, 1♣, X, XX, P, P, 1♥, 1♠, P, 2♣, P, P, 2♥, ...","[♣7, ♣A, ♣5, ♣8, ♥2, ♥7, ♥Q, ♥K, ♠2, ♠T, ♠K, ♠...",1,0,0,2♥S-2,False,"{'Event': '', 'Site': '', 'Date': '2023.12.15'...",,,2023.12.15,BENCAM22,WBridge5,BENCAM22,WBridge5,IMP,97,Closed,NS -100
2,N:T4.K62.KQ985.T54 J2.T9875.J4.AQ82 A73.AQJ43....,"[P, 1♥, 1♠, 2♥, P, P, 2♠, P, 3♠, P, P, P]","[♥6, ♥5, ♥A, ♠6, ♠8, ♠4, ♠J, ♠A, ♠3, ♠K, ♠T, ♠...",2,1,2,3♠W+1,False,"{'Event': '', 'Site': '', 'Date': '2023.12.15'...",,,2023.12.15,WBridge5,BENCAM22,WBridge5,BENCAM22,IMP,df,Open,EW 170
3,N:T4.K62.KQ985.T54 J2.T9875.J4.AQ82 A73.AQJ43....,"[P, P, 1♠, P, 1NT, 2♥, 2♠, 3♥, 3♠, P, 4♠, P, P...","[♥2, ♥7, ♥A, ♠6, ♠8, ♠4, ♠J, ♠3, ♦4, ♦2, ♦7, ♦...",2,1,2,4♠W+1,False,"{'Event': '', 'Site': '', 'Date': '2023.12.15'...",,,2023.12.15,BENCAM22,WBridge5,BENCAM22,WBridge5,IMP,97,Closed,EW 450
4,N:JT6.AK.972.T9754 K954.T3.QJ654.A6 AQ32.Q986....,"[1♣, 1♥, 2♣, P, 3♣, P, P, P]","[♦A, ♦2, ♦4, ♦3, ♦K, ♦7, ♦5, ♣3, ♥6, ♥2, ♥K, ♥...",3,2,3,3♣S+2,False,"{'Event': '', 'Site': '', 'Date': '2023.12.15'...",,,2023.12.15,WBridge5,BENCAM22,WBridge5,BENCAM22,IMP,df,Open,NS 150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,N:T732.KJT9.K6.KT7 A.74.AQJ972.AJ42 KQJ985.AQ8...,"[1♦, 2♦!*, P, 4♥, 5♦, X, P, P, P]","[♠K, ♠4, ♠7, ♠A, ♦A, ♦5, ♦3, ♦6, ♦2, ♦4, ♦8, ♦...",158,1,0,5♦Ex-2,False,"{'Event': '', 'Site': '', 'Date': '2023.12.15'...",,,2023.12.15,BENCAM22,WBridge5,BENCAM22,WBridge5,IMP,97,Closed,EW -300
316,N:QT2.QJ9.AKJ.AJ65 K84.AT85.842.Q97 A95.6432.T...,"[P, P, 1♣, P, 1♥, P, 2NT, P, 3NT, P, P, P]","[♥5, ♥4, ♥K, ♥9, ♥7, ♥Q, ♥A, ♥6, ♥T, ♥3, ♣2, ♥...",159,2,2,3NTN-3,False,"{'Event': '', 'Site': '', 'Date': '2023.12.15'...",,,2023.12.15,WBridge5,BENCAM22,WBridge5,BENCAM22,IMP,df,Open,NS -300
317,N:QT2.QJ9.AKJ.AJ65 K84.AT85.842.Q97 A95.6432.T...,"[P, P, 1NT, P, P, P]","[♥5, ♥2, ♥K, ♥9, ♥7, ♥Q, ♥A, ♥3, ♥T, ♥4, ♣4, ♥...",159,2,2,1NTN+3,False,"{'Event': '', 'Site': '', 'Date': '2023.12.15'...",,,2023.12.15,BENCAM22,WBridge5,BENCAM22,WBridge5,IMP,97,Closed,NS 180
318,N:843.9765.A73.AK4 T65.KQ82.Q52.T93 AK.AJT.986...,"[P, P, P, 1♦, 1♠, X, 2♠, P, P, X, P, 2NT, P, P...","[♠Q, ♠4, ♠T, ♠A, ♣2, ♣5, ♣K, ♣3, ♥7, ♥8, ♥J, ♥...",160,3,3,2NTS+2,False,"{'Event': '', 'Site': '', 'Date': '2023.12.15'...",,,2023.12.15,WBridge5,BENCAM22,WBridge5,BENCAM22,IMP,df,Open,NS 180


In [7]:
# calculate double dummy and par
deals = df['deal']
batch_size = 40
t_t = []
tables = []
b_ptr = 0
for b in range(0,len(deals),batch_size):
    batch_tables = calc_all_tables(deals[b:min(b+batch_size,len(deals))])
    tables.extend(batch_tables)
    batch_t_t = (tt._data.resTable for tt in batch_tables)
    t_t.extend(batch_t_t)
    b_ptr += b

assert len(deals) == len(t_t) == len(tables)

In [8]:
# display a few hands and double dummy tables
dd_tricks_rows = []
max_display = 4
for ii,(dd,sd,tt) in enumerate(zip(deals,t_t,tables)):
    if ii < max_display:
        print(f"Deal: {ii+1}")
        dd.pprint()
        print()
        tt.pprint()
        print(tuple(tuple(sd[suit][direction] for suit in suit_order) for direction in direction_order))
        print()

Deal: 1
              T5
              982
              874
              AQ632
Q8762                       K43
KJ54                        73
A93                         KQ5
7                           KJT54
              AJ9
              AQT6
              JT62
              98

     ♣  ♦  ♥  ♠ NT
  N  5  5  5  4  5
  S  5  6  6  4  5
  E  8  7  7  9  8
  W  8  7  7  9  8
((5, 5, 5, 4, 5), (5, 6, 6, 4, 5), (8, 7, 7, 9, 8), (8, 7, 7, 9, 8))

Deal: 2
              T5
              982
              874
              AQ632
Q8762                       K43
KJ54                        73
A93                         KQ5
7                           KJT54
              AJ9
              AQT6
              JT62
              98

     ♣  ♦  ♥  ♠ NT
  N  5  5  5  4  5
  S  5  6  6  4  5
  E  8  7  7  9  8
  W  8  7  7  9  8
((5, 5, 5, 4, 5), (5, 6, 6, 4, 5), (8, 7, 7, 9, 8), (8, 7, 7, 9, 8))

Deal: 3
              T4
              K62
              KQ985
              T54
KQ9865               

In [9]:
# create dataframe of par scores (double dummy).
pars = [par(tt, b, 0) for tt,b in zip(tables,df['board_num'])] # middle arg is board (if int) otherwise enum vul.
par_scores_ns = [parlist.score for parlist in pars]
par_scores_ew = [-score for score in par_scores_ns]
par_contracts = [[str(contract.level)+'SHDCN'[int(contract.denom)]+contract.declarer.abbr+contract.penalty.abbr+' '+str(contract.result) for contract in parlist] for parlist in pars]
par_df = pd.DataFrame({'Par_NS':par_scores_ns,'Par_EW':par_scores_ew,'Par_Contracts_Result':par_contracts})
par_df

,Par_NS,Par_EW,Par_Contracts_Result
0,-140,140,"[1SE 2, 1SW 2]"
1,-140,140,"[1SE 2, 1SW 2]"
2,-420,420,"[4SE 0, 4SW 0]"
3,-420,420,"[4SE 0, 4SW 0]"
4,400,-400,"[5CN 0, 5CS 0]"
...,...,...,...
315,450,-450,"[5SN 0, 5HN 0]"
316,630,-630,"[3NN 1, 3NS 1]"
317,630,-630,"[3NN 1, 3NS 1]"
318,430,-430,"[3NN 1, 3NS 1]"


In [10]:
# create dataframe of double dummy tricks per direction and suit.
dd_tricks_rows = [[sd[suit][direction] for direction in direction_order for suit in suit_order] for sd in t_t]
dd_tricks_df = pd.DataFrame(dd_tricks_rows,columns=['_'.join(['DD_Tricks',d,s]) for d in 'NSEW' for s in 'CDHSN'])
dd_tricks_df

,DD_Tricks_N_C,DD_Tricks_N_D,DD_Tricks_N_H,DD_Tricks_N_S,DD_Tricks_N_N,DD_Tricks_S_C,DD_Tricks_S_D,DD_Tricks_S_H,DD_Tricks_S_S,DD_Tricks_S_N,DD_Tricks_E_C,DD_Tricks_E_D,DD_Tricks_E_H,DD_Tricks_E_S,DD_Tricks_E_N,DD_Tricks_W_C,DD_Tricks_W_D,DD_Tricks_W_H,DD_Tricks_W_S,DD_Tricks_W_N
0,5,5,5,4,5,5,6,6,4,5,8,7,7,9,8,8,7,7,9,8
1,5,5,5,4,5,5,6,6,4,5,8,7,7,9,8,8,7,7,9,8
2,2,7,7,3,5,2,7,7,3,5,11,5,6,10,5,11,5,6,10,5
3,2,7,7,3,5,2,7,7,3,5,11,5,6,10,5,11,5,6,10,5
4,11,5,7,8,7,11,5,7,8,7,2,8,6,4,6,2,8,5,4,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,3,4,11,11,6,3,3,10,10,0,9,8,2,2,3,9,8,2,2,3
316,9,9,9,9,10,9,9,9,9,10,3,4,3,4,3,3,4,4,4,3
317,9,9,9,9,10,9,9,9,9,10,3,4,3,4,3,3,4,4,4,3
318,10,10,10,7,10,10,10,10,7,10,3,3,3,6,3,3,3,3,6,3


In [11]:
# create dataframe of double dummy scores per direction and suit.
def Tricks_To_Score(sd):
    return [Contract(level=level,denom=suit,declarer=direction,penalty=Penalty.passed if sd[suit][direction]-6-level>=0 else Penalty.doubled,result=sd[suit][direction]-6-level).score(0) for direction in direction_order for suit in suit_order for level in range(1,8)]

direction_order = [0,2,1,3] # NSEW order
suit_order = [3,2,1,0,4] # SHDCN order?
dd_score_rows = [Tricks_To_Score(sd) for sd in t_t]
dd_score_df = pd.DataFrame(dd_score_rows,columns=['_'.join(['DD_Score',str(l)+s,d]) for d in 'NSEW' for s in 'CDHSN' for l in range(1,8)])
dd_score_df


,DD_Score_1C_N,DD_Score_2C_N,DD_Score_3C_N,DD_Score_4C_N,DD_Score_5C_N,DD_Score_6C_N,DD_Score_7C_N,DD_Score_1D_N,DD_Score_2D_N,DD_Score_3D_N,DD_Score_4D_N,DD_Score_5D_N,DD_Score_6D_N,DD_Score_7D_N,DD_Score_1H_N,DD_Score_2H_N,DD_Score_3H_N,DD_Score_4H_N,DD_Score_5H_N,DD_Score_6H_N,DD_Score_7H_N,DD_Score_1S_N,DD_Score_2S_N,DD_Score_3S_N,DD_Score_4S_N,DD_Score_5S_N,DD_Score_6S_N,DD_Score_7S_N,DD_Score_1N_N,DD_Score_2N_N,DD_Score_3N_N,DD_Score_4N_N,DD_Score_5N_N,DD_Score_6N_N,DD_Score_7N_N,DD_Score_1C_S,DD_Score_2C_S,DD_Score_3C_S,DD_Score_4C_S,DD_Score_5C_S,...,DD_Score_3N_E,DD_Score_4N_E,DD_Score_5N_E,DD_Score_6N_E,DD_Score_7N_E,DD_Score_1C_W,DD_Score_2C_W,DD_Score_3C_W,DD_Score_4C_W,DD_Score_5C_W,DD_Score_6C_W,DD_Score_7C_W,DD_Score_1D_W,DD_Score_2D_W,DD_Score_3D_W,DD_Score_4D_W,DD_Score_5D_W,DD_Score_6D_W,DD_Score_7D_W,DD_Score_1H_W,DD_Score_2H_W,DD_Score_3H_W,DD_Score_4H_W,DD_Score_5H_W,DD_Score_6H_W,DD_Score_7H_W,DD_Score_1S_W,DD_Score_2S_W,DD_Score_3S_W,DD_Score_4S_W,DD_Score_5S_W,DD_Score_6S_W,DD_Score_7S_W,DD_Score_1N_W,DD_Score_2N_W,DD_Score_3N_W,DD_Score_4N_W,DD_Score_5N_W,DD_Score_6N_W,DD_Score_7N_W
0,-300,-500,-800,-1100,-1400,-1700,-2000,-300,-500,-800,-1100,-1400,-1700,-2000,-300,-500,-800,-1100,-1400,-1700,-2000,-500,-800,-1100,-1400,-1700,-2000,-2300,-300,-500,-800,-1100,-1400,-1700,-2000,-300,-500,-800,-1100,-1400,...,-100,-300,-500,-800,-1100,90,90,-100,-300,-500,-800,-1100,70,-100,-300,-500,-800,-1100,-1400,80,-100,-300,-500,-800,-1100,-1400,140,140,140,-100,-300,-500,-800,120,120,-100,-300,-500,-800,-1100
1,-300,-500,-800,-1100,-1400,-1700,-2000,-300,-500,-800,-1100,-1400,-1700,-2000,-300,-500,-800,-1100,-1400,-1700,-2000,-500,-800,-1100,-1400,-1700,-2000,-2300,-300,-500,-800,-1100,-1400,-1700,-2000,-300,-500,-800,-1100,-1400,...,-100,-300,-500,-800,-1100,90,90,-100,-300,-500,-800,-1100,70,-100,-300,-500,-800,-1100,-1400,80,-100,-300,-500,-800,-1100,-1400,140,140,140,-100,-300,-500,-800,120,120,-100,-300,-500,-800,-1100
2,-1100,-1400,-1700,-2000,-2300,-2600,-2900,70,-100,-300,-500,-800,-1100,-1400,80,-100,-300,-500,-800,-1100,-1400,-800,-1100,-1400,-1700,-2000,-2300,-2600,-300,-500,-800,-1100,-1400,-1700,-2000,-1100,-1400,-1700,-2000,-2300,...,-800,-1100,-1400,-1700,-2000,150,150,150,150,400,-100,-300,-300,-500,-800,-1100,-1400,-1700,-2000,-100,-300,-500,-800,-1100,-1400,-1700,170,170,170,420,-100,-300,-500,-300,-500,-800,-1100,-1400,-1700,-2000
3,-1100,-1400,-1700,-2000,-2300,-2600,-2900,70,-100,-300,-500,-800,-1100,-1400,80,-100,-300,-500,-800,-1100,-1400,-800,-1100,-1400,-1700,-2000,-2300,-2600,-300,-500,-800,-1100,-1400,-1700,-2000,-1100,-1400,-1700,-2000,-2300,...,-800,-1100,-1400,-1700,-2000,150,150,150,150,400,-100,-300,-300,-500,-800,-1100,-1400,-1700,-2000,-100,-300,-500,-800,-1100,-1400,-1700,170,170,170,420,-100,-300,-500,-300,-500,-800,-1100,-1400,-1700,-2000
4,150,150,150,150,400,-100,-300,-300,-500,-800,-1100,-1400,-1700,-2000,80,-100,-300,-500,-800,-1100,-1400,110,110,-100,-300,-500,-800,-1100,90,-100,-300,-500,-800,-1100,-1400,150,150,150,150,400,...,-500,-800,-1100,-1400,-1700,-1100,-1400,-1700,-2000,-2300,-2600,-2900,90,90,-100,-300,-500,-800,-1100,-300,-500,-800,-1100,-1400,-1700,-2000,-500,-800,-1100,-1400,-1700,-2000,-2300,-100,-300,-500,-800,-1100,-1400,-1700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,-800,-1100,-1400,-1700,-2000,-2300,-2600,-500,-800,-1100,-1400,-1700,-2000,-2300,200,200,200,450,450,-100,-300,200,200,200,450,450,-100,-300,-100,-300,-500,-800,-1100,-1400,-1700,-800,-1100,-1400,-1700,-2000,...,-1400,-1700,-2000,-2300,-2600,110,110,110,-100,-300,-500,-800,90,90,-100,-300,-500,-800,-1100,-1100,-1400,-1700,-2000,-2300,-2600,-2900,-1100,-1400,-1700,-2000,-2300,-2600,-2900,-800,-1100,-1400,-170

In [12]:
# functions to calculate single dummy probabilities.

# todo: obsolete these constants?
CDHS = 'CDHS' # string ordered by suit rank - low to high
CDHSN = CDHS+'N' # string ordered by strain
NSHDC = 'NSHDC' # order by highest score value. useful for idxmax(). coincidentally reverse of CDHSN.
SHDC = 'SHDC' # Hands, PBN, board_record_string (brs) ordering
NSEW = 'NSEW' # double dummy solver ordering
NESW = 'NESW' # Hands and PBN order
NWES = 'NWES' # board_record_string (brs) ordering
SHDCN = 'SHDCN' # ordering used by dds

# todo: could save a couple seconds by creating dict of deals
def calc_double_dummy_deals(deals, batch_size=40):
    t_t = []
    tables = []
    for b in range(0,len(deals),batch_size):
        batch_tables = calc_all_tables(deals[b:min(b+batch_size,len(deals))])
        tables.extend(batch_tables)
        batch_t_t = (tt._data.resTable for tt in batch_tables)
        t_t.extend(batch_t_t)
    assert len(t_t) == len(tables)
    return deals, t_t, tables
    return df

def constraints(deal):
    return True

def generate_single_dummy_deals(predeal_string, produce, env=dict(), max_attempts=1000000, seed=None, show_progress=True, strict=True, swapping=0):
    
    predeal = Deal(predeal_string)

    deals_t = generate_deals(
        constraints,
        predeal=predeal,
        swapping=swapping,
        show_progress=show_progress,
        produce=produce,
        seed=seed,
        max_attempts=max_attempts,
        env=env,
        strict=strict
        )

    deals = tuple(deals_t) # create a tuple before interop memory goes wonky
    
    return calc_double_dummy_deals(deals)

def calculate_single_dummy_probabilities(deal, produce=100):

    ns_ew_rows = {}
    for ns_ew in ['NS','EW']:
        s = deal[2:].split()
        if ns_ew == 'NS':
            s[1] = '...'
            s[3] = '...'
        else:
            s[0] = '...'
            s[2] = '...'
        predeal_string = 'N:'+' '.join(s)
        #print_to_log(f"predeal:{predeal_string}")

        d_t, t_t, tables = generate_single_dummy_deals(predeal_string, produce, show_progress=False)

        rows = []
        max_display = 4 # pprint only the first n generated deals
        direction_order = [0,2,1,3] # NSEW order
        suit_order = [3,2,1,0,4] # SHDCN order?
        for ii,(dd,sd,tt) in enumerate(zip(d_t,t_t,tables)):
            # if ii < max_display:
                # print_to_log(f"Deal:{ii+1} Fixed:{ns_ew} Generated:{ii+1}/{produce}")
                # dd.pprint()
                # print_to_log()
                # tt.pprint()
                # print_to_log()
            nswe_flat_l = [sd[suit][direction] for direction in direction_order for suit in suit_order]
            rows.append([dd.to_pbn()]+nswe_flat_l)

        dd_df = pd.DataFrame(rows,columns=['Deal']+[d+s for d in NSEW for s in CDHSN])
        for d in NSEW:
            for s in SHDCN:
                ns_ew_rows[(ns_ew,d,s)] = dd_df[d+s].value_counts(normalize=True).reindex(range(14), fill_value=0).tolist() # ['Fixed_Direction','Direction_Declarer','Suit']+['SD_Prob_Take_'+str(n) for n in range(14)]
    
    return ns_ew_rows


def append_single_dummy_results(pbns,sd_cache_d,produce=100):

    for pbn in pbns:
        if pbn not in sd_cache_d:
            sd_cache_d[pbn] = calculate_single_dummy_probabilities(pbn, produce) # all combinations of declarer pair direction, declarer direciton, suit, tricks taken
    return sd_cache_d


In [13]:
# takes 1000 seconds for 100 sd calcs, or 10 sd calcs per second.
sd_cache_d = {}
pbns = [str(pbn) for pbn in deals]
for i,pbn in enumerate(pbns):
    print(f"{i} of {len(pbns)} boards. pbn:{pbn}")
    if pbn not in sd_cache_d:
        sd_cache_d[pbn] = calculate_single_dummy_probabilities(pbn, sd_productions) # all combinations of declarer pair direction, declarer direciton, suit, tricks taken


0 of 320 boards. pbn:N:T5.982.874.AQ632 K43.73.KQ5.KJT54 AJ9.AQT6.JT62.98 Q8762.KJ54.A93.7
1 of 320 boards. pbn:N:T5.982.874.AQ632 K43.73.KQ5.KJT54 AJ9.AQT6.JT62.98 Q8762.KJ54.A93.7
2 of 320 boards. pbn:N:T4.K62.KQ985.T54 J2.T9875.J4.AQ82 A73.AQJ43.T32.96 KQ9865..A76.KJ73
3 of 320 boards. pbn:N:T4.K62.KQ985.T54 J2.T9875.J4.AQ82 A73.AQJ43.T32.96 KQ9865..A76.KJ73
4 of 320 boards. pbn:N:JT6.AK.972.T9754 K954.T3.QJ654.A6 AQ32.Q986.3.K832 87.J7542.AKT8.QJ
5 of 320 boards. pbn:N:JT6.AK.972.T9754 K954.T3.QJ654.A6 AQ32.Q986.3.K832 87.J7542.AKT8.QJ
6 of 320 boards. pbn:N:.K964.KQ93.KJ532 96543.5.J74.AT94 87.J873.T852.Q87 AKQJT2.AQT2.A6.6
7 of 320 boards. pbn:N:.K964.KQ93.KJ532 96543.5.J74.AT94 87.J873.T852.Q87 AKQJT2.AQT2.A6.6
8 of 320 boards. pbn:N:T5.AK94.QT3.AKJ3 96.QJT3.976.8654 AJ82.872.K85.T92 KQ743.65.AJ42.Q7
9 of 320 boards. pbn:N:T5.AK94.QT3.AKJ3 96.QJT3.976.8654 AJ82.872.K85.T92 KQ743.65.AJ42.Q7
10 of 320 boards. pbn:N:AKJ.AT943.Q972.3 QT84.J72..KQJT42 965.K6.AK654.A98 732.Q85.JT83.76

In [14]:
# calculate single dummy trick taking probability distribution
sd_probs_d = defaultdict(list)
for pbn in pbns:
    #d['PBN'].append(pbn)
    v = sd_cache_d[pbn]
    print(pbn,v)
    for (pair_direction,declarer_direction,suit),tricks in v.items():
        for i,t in enumerate(tricks):
            sd_probs_d['_'.join(['Probs',pair_direction,declarer_direction,suit,str(i)])].append(t)
print(sd_probs_d)
sd_probs_df = pd.DataFrame(sd_probs_d)
sd_probs_df

N:T5.982.874.AQ632 K43.73.KQ5.KJT54 AJ9.AQT6.JT62.98 Q8762.KJ54.A93.7 {('NS', 'N', 'S'): [0.0, 0.0, 0.0, 0.07, 0.4, 0.36, 0.16, 0.01, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], ('NS', 'N', 'H'): [0.0, 0.0, 0.0, 0.0, 0.01, 0.11, 0.36, 0.44, 0.07, 0.01, 0.0, 0.0, 0.0, 0.0], ('NS', 'N', 'D'): [0.0, 0.0, 0.0, 0.0, 0.03, 0.24, 0.39, 0.3, 0.04, 0.0, 0.0, 0.0, 0.0, 0.0], ('NS', 'N', 'C'): [0.0, 0.0, 0.0, 0.0, 0.02, 0.23, 0.33, 0.31, 0.11, 0.0, 0.0, 0.0, 0.0, 0.0], ('NS', 'N', 'N'): [0.0, 0.0, 0.0, 0.01, 0.07, 0.21, 0.39, 0.28, 0.04, 0.0, 0.0, 0.0, 0.0, 0.0], ('NS', 'S', 'S'): [0.0, 0.0, 0.0, 0.04, 0.37, 0.41, 0.17, 0.01, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], ('NS', 'S', 'H'): [0.0, 0.0, 0.0, 0.0, 0.0, 0.09, 0.41, 0.38, 0.11, 0.01, 0.0, 0.0, 0.0, 0.0], ('NS', 'S', 'D'): [0.0, 0.0, 0.0, 0.0, 0.03, 0.23, 0.41, 0.29, 0.04, 0.0, 0.0, 0.0, 0.0, 0.0], ('NS', 'S', 'C'): [0.0, 0.0, 0.0, 0.0, 0.01, 0.17, 0.37, 0.33, 0.12, 0.0, 0.0, 0.0, 0.0, 0.0], ('NS', 'S', 'N'): [0.0, 0.0, 0.0, 0.0, 0.08, 0.22, 0.37, 0.28, 0.05, 0.0, 

,Probs_NS_N_S_0,Probs_NS_N_S_1,Probs_NS_N_S_2,Probs_NS_N_S_3,Probs_NS_N_S_4,Probs_NS_N_S_5,Probs_NS_N_S_6,Probs_NS_N_S_7,Probs_NS_N_S_8,Probs_NS_N_S_9,Probs_NS_N_S_10,Probs_NS_N_S_11,Probs_NS_N_S_12,Probs_NS_N_S_13,Probs_NS_N_H_0,Probs_NS_N_H_1,Probs_NS_N_H_2,Probs_NS_N_H_3,Probs_NS_N_H_4,Probs_NS_N_H_5,Probs_NS_N_H_6,Probs_NS_N_H_7,Probs_NS_N_H_8,Probs_NS_N_H_9,Probs_NS_N_H_10,Probs_NS_N_H_11,Probs_NS_N_H_12,Probs_NS_N_H_13,Probs_NS_N_D_0,Probs_NS_N_D_1,Probs_NS_N_D_2,Probs_NS_N_D_3,Probs_NS_N_D_4,Probs_NS_N_D_5,Probs_NS_N_D_6,Probs_NS_N_D_7,Probs_NS_N_D_8,Probs_NS_N_D_9,Probs_NS_N_D_10,Probs_NS_N_D_11,...,Probs_EW_W_D_2,Probs_EW_W_D_3,Probs_EW_W_D_4,Probs_EW_W_D_5,Probs_EW_W_D_6,Probs_EW_W_D_7,Probs_EW_W_D_8,Probs_EW_W_D_9,Probs_EW_W_D_10,Probs_EW_W_D_11,Probs_EW_W_D_12,Probs_EW_W_D_13,Probs_EW_W_C_0,Probs_EW_W_C_1,Probs_EW_W_C_2,Probs_EW_W_C_3,Probs_EW_W_C_4,Probs_EW_W_C_5,Probs_EW_W_C_6,Probs_EW_W_C_7,Probs_EW_W_C_8,Probs_EW_W_C_9,Probs_EW_W_C_10,Probs_EW_W_C_11,Probs_EW_W_C_12,Probs_EW_W_C_13,Probs_EW_W_N_0,Probs_EW_W_N_1,Probs_EW_W_N_2,Probs_EW_W_N_3,Probs_EW_W_N_4,Probs_EW_W_N_5,Probs_EW_W_N_6,Probs_EW_W_N_7,Probs_EW_W_N_8,Probs_EW_W_N_9,Probs_EW_W_N_10,Probs_EW_W_N_11,Probs_EW_W_N_12,Probs_EW_W_N_13
0,0.0,0.0,0.00,0.07,0.40,0.36,0.16,0.01,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.11,0.36,0.44,0.07,0.01,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.03,0.24,0.39,0.30,0.04,0.00,0.00,0.0,...,0.00,0.00,0.00,0.06,0.28,0.57,0.09,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.05,0.21,0.43,0.31,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.02,0.20,0.33,0.3,0.15,0.0,0.0,0.0,0.0
1,0.0,0.0,0.00,0.07,0.40,0.36,0.16,0.01,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.11,0.36,0.44,0.07,0.01,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.03,0.24,0.39,0.30,0.04,0.00,0.00,0.0,...,0.00,0.00,0.00,0.06,0.28,0.57,0.09,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.05,0.21,0.43,0.31,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.02,0.20,0.33,0.3,0.15,0.0,0.0,0.0,0.0
2,0.0,0.0,0.14,0.56,0.30,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.15,0.37,0.48,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.02,0.08,0.36,0.54,0.00,0.0,...,0.01,0.05,0.26,0.45,0.23,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.04,0.17,0.26,0.53,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.96,0.04,0.00,0.0,0.00,0.0,0.0,0.0,0.0
3,0.0,0.0,0.14,0.56,0.30,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.15,0.37,0.48,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.02,0.08,0.36,0.54,0.00,0.0,...,0.01,0.05,0.26,0.45,0.23,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.04,0.17,0.26,0.53,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.96,0.04,0.00,0.0,0.00,0.0,0.0,0.0,0.0
4,0.0,0.0,0.00,0.00,0.00,0.04,0.07,0.24,0.41,0.24,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.20,0.39,0.31,0.07,0.02,0.00,0.00,0.0,0.0,0.00,0.0,0.01,0.19,0.39,0.25,0.13,0.03,0.00,0.00,0.00,0.0,...,0.00,0.00,0.00,0.00,0.00,0.01,0.28,0.53,0.18,0.00,0.0,0.0,0.00,0.01,0.26,0.41,0.26,0.06,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.45,0.25,0.3,0.00,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.08,0.35,0.56,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.02,0.32,0.51,0.15,0.0,0.0,0.02,0.1,0.30,0.47,0.11,0.00,0.00,0.00,0.00,0.00,0.00,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.11,0.49,0.35,0.05,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.04,0.37,0.48,0.11,0.0,0.0,0.0,0.00,0.00,0.64,0.00,0.00,0.04,0.12,0.2,0.00,0.0,0.0,0.0,0.0
316,0.0,0.0,0.00,0.00,0.00,0.00,0.02,0.23,0.42,0.33,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.03,0.22,0.65,0.10,0.00,0.0,0.0,0.00,

In [15]:
# calculate dict of contract result scores
sd_scores_d = {}
for suit in suit_order:
    for level in range(1,8): # contract level
        for tricks in range(14):
            result = tricks-6-level
            sd_scores_d[(level,'SHDCN'[suit],tricks,False)] = Contract(level=level,denom=suit,declarer=0,penalty=Penalty.passed if result>=0 else Penalty.doubled,result=result).score(False)
            sd_scores_d[(level,'SHDCN'[suit],tricks,True)] = Contract(level=level,denom=suit,declarer=0,penalty=Penalty.passed if result>=0 else Penalty.doubled,result=result).score(True)
sd_scores_d

{(1, 'C', 0, False): -1700,
 (1, 'C', 0, True): -2000,
 (1, 'C', 1, False): -1400,
 (1, 'C', 1, True): -1700,
 (1, 'C', 2, False): -1100,
 (1, 'C', 2, True): -1400,
 (1, 'C', 3, False): -800,
 (1, 'C', 3, True): -1100,
 (1, 'C', 4, False): -500,
 (1, 'C', 4, True): -800,
 (1, 'C', 5, False): -300,
 (1, 'C', 5, True): -500,
 (1, 'C', 6, False): -100,
 (1, 'C', 6, True): -200,
 (1, 'C', 7, False): 70,
 (1, 'C', 7, True): 70,
 (1, 'C', 8, False): 90,
 (1, 'C', 8, True): 90,
 (1, 'C', 9, False): 110,
 (1, 'C', 9, True): 110,
 (1, 'C', 10, False): 130,
 (1, 'C', 10, True): 130,
 (1, 'C', 11, False): 150,
 (1, 'C', 11, True): 150,
 (1, 'C', 12, False): 170,
 (1, 'C', 12, True): 170,
 (1, 'C', 13, False): 190,
 (1, 'C', 13, True): 190,
 (2, 'C', 0, False): -2000,
 (2, 'C', 0, True): -2300,
 (2, 'C', 1, False): -1700,
 (2, 'C', 1, True): -2000,
 (2, 'C', 2, False): -1400,
 (2, 'C', 2, True): -1700,
 (2, 'C', 3, False): -1100,
 (2, 'C', 3, True): -1400,
 (2, 'C', 4, False): -800,
 (2, 'C', 4, T

In [16]:
# create score dataframe from dict
scores_d = defaultdict(list)
for suit in 'SHDCN':
    for level in range(1,8):
        for i in range(14):
            scores_d['_'.join(['Score',str(level)+suit])].append([sd_scores_d[(level,suit,i,False)],sd_scores_d[(level,suit,i,True)]])
print(scores_d)
sd_scores_df = pd.DataFrame(scores_d)
sd_scores_df.index.name = 'Taken'
sd_scores_df

defaultdict(<class 'list'>, {'Score_1S': [[-1700, -2000], [-1400, -1700], [-1100, -1400], [-800, -1100], [-500, -800], [-300, -500], [-100, -200], [80, 80], [110, 110], [140, 140], [170, 170], [200, 200], [230, 230], [260, 260]], 'Score_2S': [[-2000, -2300], [-1700, -2000], [-1400, -1700], [-1100, -1400], [-800, -1100], [-500, -800], [-300, -500], [-100, -200], [110, 110], [140, 140], [170, 170], [200, 200], [230, 230], [260, 260]], 'Score_3S': [[-2300, -2600], [-2000, -2300], [-1700, -2000], [-1400, -1700], [-1100, -1400], [-800, -1100], [-500, -800], [-300, -500], [-100, -200], [140, 140], [170, 170], [200, 200], [230, 230], [260, 260]], 'Score_4S': [[-2600, -2900], [-2300, -2600], [-2000, -2300], [-1700, -2000], [-1400, -1700], [-1100, -1400], [-800, -1100], [-500, -800], [-300, -500], [-100, -200], [420, 620], [450, 650], [480, 680], [510, 710]], 'Score_5S': [[-2900, -3200], [-2600, -2900], [-2300, -2600], [-2000, -2300], [-1700, -2000], [-1400, -1700], [-1100, -1400], [-800, -1100

,Score_1S,Score_2S,Score_3S,Score_4S,Score_5S,Score_6S,Score_7S,Score_1H,Score_2H,Score_3H,Score_4H,Score_5H,Score_6H,Score_7H,Score_1D,Score_2D,Score_3D,Score_4D,Score_5D,Score_6D,Score_7D,Score_1C,Score_2C,Score_3C,Score_4C,Score_5C,Score_6C,Score_7C,Score_1N,Score_2N,Score_3N,Score_4N,Score_5N,Score_6N,Score_7N
Taken,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,"[-1700, -2000]","[-2000, -2300]","[-2300, -2600]","[-2600, -2900]","[-2900, -3200]","[-3200, -3500]","[-3500, -3800]","[-1700, -2000]","[-2000, -2300]","[-2300, -2600]","[-2600, -2900]","[-2900, -3200]","[-3200, -3500]","[-3500, -3800]","[-1700, -2000]","[-2000, -2300]","[-2300, -2600]","[-2600, -2900]","[-2900, -3200]","[-3200, -3500]","[-3500, -3800]","[-1700, -2000]","[-2000, -2300]","[-2300, -2600]","[-2600, -2900]","[-2900, -3200]","[-3200, -3500]","[-3500, -3800]","[-1700, -2000]","[-2000, -2300]","[-2300, -2600]","[-2600, -2900]","[-2900, -3200]","[-3200, -3500]","[-3500, -3800]"
1,"[-1400, -1700]","[-1700, -2000]","[-2000, -2300]","[-2300, -2600]","[-2600, -2900]","[-2900, -3200]","[-3200, -3500]","[-1400, -1700]","[-1700, -2000]","[-2000, -2300]","[-2300, -2600]","[-2600, -2900]","[-2900, -3200]","[-3200, -3500]","[-1400, -1700]","[-1700, -2000]","[-2000, -2300]","[-2300, -2600]","[-2600, -2900]","[-2900, -3200]","[-3200, -3500]","[-1400, -1700]","[-1700, -2000]","[-2000, -2300]","[-2300, -2600]","[-2600, -2900]","[-2900, -3200]","[-3200, -3500]","[-1400, -1700]","[-1700, -2000]","[-2000, -2300]","[-2300, -2600]","[-2600, -2900]","[-2900, -3200]","[-3200, -3500]"
2,"[-1100, -1400]","[-1400, -1700]","[-1700, -2000]","[-2000, -2300]","[-2300, -2600]","[-2600, -2900]","[-2900, -3200]","[-1100, -1400]","[-1400, -1700]","[-1700, -2000]","[-2000, -2300]","[-2300, -2600]","[-2600, -2900]","[-2900, -3200]","[-1100, -1400]","[-1400, -1700]","[-1700, -2000]","[-2000, -2300]","[-2300, -2600]","[-2600, -2900]","[-2900, -3200]","[-1100, -1400]","[-1400, -1700]","[-1700, -2000]","[-2000, -2300]","[-2300, -2600]","[-2600, -2900]","[-2900, -3200]","[-1100, -1400]","[-1400, -1700]","[-1700, -2000]","[-2000, -2300]","[-2300, -2600]","[-2600, -2900]","[-2900, -3200]"
3,"[-800, -1100]","[-1100, -1400]","[-1400, -1700]","[-1700, -2000]","[-2000, -2300]","[-2300, -2600]","[-2600, -2900]","[-800, -1100]","[-1100, -1400]","[-1400, -1700]","[-1700, -2000]","[-2000, -2300]","[-2300, -2600]","[-2600, -2900]","[-800, -1100]","[-1100, -1400]","[-1400, -1700]","[-1700, -2000]","[-2000, -2300]","[-2300, -2600]","[-2600, -2900]","[-800, -1100]","[-1100, -1400]","[-1400, -1700]","[-1700, -2000]","[-2000, -2300]","[-2300, -2600]","[-2600, -2900]","[-800, -1100]","[-1100, -1400]","[-1400, -1700]","[-1700, -2000]","[-2000, -2300]","[-2300, -2600]","[-2600, -2900]"
4,"[-500, -800]","[-800, -1100]","[-1100, -1400]","[-1400, -1700]","[-1700, -2000]","[-2000, -2300]","[-2300, -2600]","[-500, -800]","[-800, -1100]","[-1100, -1400]","[-1400, -1700]","[-1700, -2000]","[-2000, -2300]","[-2300, -2600]","[-500, -800]","[-800, -1100]","[-1100, -1400]","[-1400, -1700]","[-1700, -2000]","[-2000, -2300]","[-2300, -2600]","[-500, -800]","[-800, -1100]","[-1100, -1400]","[-1400, -1700]","[-1700, -2000]","[-2000, -2300]","[-2300, -2600]","[-500, -800]","[-800, -1100]","[-1100, -1400]","[-1400, -1700]","[-1700, -2000]","[-2000, -2300]","[-2300, -2600]"
5,"[-300, -500]","[-500, -800]","[-800, -1100]","[-1100, -1400]","[-1400, -1700]","[-1700, -2000]","[-2000, -2300]","[-300, -500]","[-500, -800]","[-800, -1100]","[-1100, -1400]","[-1400, -1700]","[-1700, -2000]","[-2000, -2300]","[-300, -500]","[-500, -800]","[-800, -1100]","[-1100, -1400]","[-1400, -1700]","[-1700, -2000]","[-2000, -2300]","[-300, -500]","[-500, -800]","[-800, -1100]","[-1100, -1400]","[-1400, -1700]","[-1700, -2000]","[-2000, -2300]","[-300, -500]","[-500, -800]","[-800, -1100]","[-1100, -1400]","[-1400, -1700]","[-1700, -2000]","[-2000, -2300]"
6,"[-100, -200]","[-300, -500]","[-500, -800]","[-800, -1100]","[-1100, -1400]","[-1400, -1700]","[-1700, 

In [17]:
# create dict of expected values (probability * score)
exp_d = defaultdict(list)
pbn_vul = zip(pbns,df['_vul'])
for pbn,vul in pbn_vul:
    #print(pbn,vul)
    for (pair_direction,declarer_direction,suit),probs in sd_cache_d[pbn].items():
        is_vul = vul == 1 or (declarer_direction in 'NS' and vul == 2) or (declarer_direction in 'EW' and vul == 3)
        #print(pair_direction,declarer_direction,suit,probs,is_vul)
        for level in range(1,8):
            #print(scores_d['_'.join(['Score',str(level)+suit])][is_vul])
            exp_d['_'.join(['Exp',pair_direction,declarer_direction,suit,str(level)])].append(sum([prob*score[is_vul] for prob,score in zip(probs,scores_d['_'.join(['Score',str(level)+suit])])]))
        #print(exp_d)
#print(exp_d)
sd_exp_df = pd.DataFrame(exp_d)
sd_exp_df

,Exp_NS_N_S_1,Exp_NS_N_S_2,Exp_NS_N_S_3,Exp_NS_N_S_4,Exp_NS_N_S_5,Exp_NS_N_S_6,Exp_NS_N_S_7,Exp_NS_N_H_1,Exp_NS_N_H_2,Exp_NS_N_H_3,Exp_NS_N_H_4,Exp_NS_N_H_5,Exp_NS_N_H_6,Exp_NS_N_H_7,Exp_NS_N_D_1,Exp_NS_N_D_2,Exp_NS_N_D_3,Exp_NS_N_D_4,Exp_NS_N_D_5,Exp_NS_N_D_6,Exp_NS_N_D_7,Exp_NS_N_C_1,Exp_NS_N_C_2,Exp_NS_N_C_3,Exp_NS_N_C_4,Exp_NS_N_C_5,Exp_NS_N_C_6,Exp_NS_N_C_7,Exp_NS_N_N_1,Exp_NS_N_N_2,Exp_NS_N_N_3,Exp_NS_N_N_4,Exp_NS_N_N_5,Exp_NS_N_N_6,Exp_NS_N_N_7,Exp_NS_S_S_1,Exp_NS_S_S_2,Exp_NS_S_S_3,Exp_NS_S_S_4,Exp_NS_S_S_5,...,Exp_EW_E_N_3,Exp_EW_E_N_4,Exp_EW_E_N_5,Exp_EW_E_N_6,Exp_EW_E_N_7,Exp_EW_W_S_1,Exp_EW_W_S_2,Exp_EW_W_S_3,Exp_EW_W_S_4,Exp_EW_W_S_5,Exp_EW_W_S_6,Exp_EW_W_S_7,Exp_EW_W_H_1,Exp_EW_W_H_2,Exp_EW_W_H_3,Exp_EW_W_H_4,Exp_EW_W_H_5,Exp_EW_W_H_6,Exp_EW_W_H_7,Exp_EW_W_D_1,Exp_EW_W_D_2,Exp_EW_W_D_3,Exp_EW_W_D_4,Exp_EW_W_D_5,Exp_EW_W_D_6,Exp_EW_W_D_7,Exp_EW_W_C_1,Exp_EW_W_C_2,Exp_EW_W_C_3,Exp_EW_W_C_4,Exp_EW_W_C_5,Exp_EW_W_C_6,Exp_EW_W_C_7,Exp_EW_W_N_1,Exp_EW_W_N_2,Exp_EW_W_N_3,Exp_EW_W_N_4,Exp_EW_W_N_5,Exp_EW_W_N_6,Exp_EW_W_N_7
0,-379.2,-626.0,-909.0,-1208.0,-1508.0,-1808.0,-2108.0,-29.7,-205.9,-416.6,-665.0,-957.0,-1256.0,-1556.0,-101.4,-287.4,-514.0,-780.0,-1076.0,-1376.0,-1676.0,-80.4,-251.1,-475.0,-733.0,-1022.0,-1322.0,-1622.0,-115.0,-312.2,-542.0,-810.0,-1106.0,-1406.0,-1706.0,-356.2,-597.0,-879.0,-1178.0,-1478.0,...,-227.0,-489.0,-749.0,-1037.0,-1337.0,124.4,111.6,19.9,-179.0,-402.0,-646.0,-941.0,4.5,-162.9,-371.0,-607.0,-896.0,-1196.0,-1496.0,2.0,-162.9,-368.0,-602.0,-893.0,-1193.0,-1493.0,22.0,-103.1,-305.0,-531.0,-800.0,-1100.0,-1400.0,62.2,-44.5,-185.0,-452.0,-707.0,-992.0,-1292.0
1,-379.2,-626.0,-909.0,-1208.0,-1508.0,-1808.0,-2108.0,-29.7,-205.9,-416.6,-665.0,-957.0,-1256.0,-1556.0,-101.4,-287.4,-514.0,-780.0,-1076.0,-1376.0,-1676.0,-80.4,-251.1,-475.0,-733.0,-1022.0,-1322.0,-1622.0,-115.0,-312.2,-542.0,-810.0,-1106.0,-1406.0,-1706.0,-356.2,-597.0,-879.0,-1178.0,-1478.0,...,-227.0,-489.0,-749.0,-1037.0,-1337.0,124.4,111.6,19.9,-179.0,-402.0,-646.0,-941.0,4.5,-162.9,-371.0,-607.0,-896.0,-1196.0,-1496.0,2.0,-162.9,-368.0,-602.0,-893.0,-1193.0,-1493.0,22.0,-103.1,-305.0,-531.0,-800.0,-1100.0,-1400.0,62.2,-44.5,-185.0,-452.0,-707.0,-992.0,-1292.0
2,-1052.0,-1352.0,-1652.0,-1952.0,-2252.0,-2552.0,-2852.0,119.9,77.9,-81.8,-401.0,-701.0,-1001.0,-1301.0,93.4,65.8,-68.6,-374.0,-674.0,-974.0,-1274.0,-1007.0,-1307.0,-1607.0,-1907.0,-2207.0,-2507.0,-2807.0,-209.0,-509.0,-809.0,-1109.0,-1409.0,-1709.0,-2009.0,-1052.0,-1352.0,-1652.0,-1952.0,-2252.0,...,-788.0,-1088.0,-1388.0,-1688.0,-1988.0,166.1,166.1,166.1,352.4,-126.0,-326.0,-539.0,-301.0,-528.0,-797.0,-1097.0,-1397.0,-1697.0,-1997.0,-339.0,-571.0,-848.0,-1148.0,-1448.0,-1748.0,-2048.0,135.6,135.6,128.0,84.3,115.0,-248.0,-469.0,-292.0,-492.0,-788.0,-1088.0,-1388.0,-1688.0,-1988.0
3,-1052.0,-1352.0,-1652.0,-1952.0,-2252.0,-2552.0,-2852.0,119.9,77.9,-81.8,-401.0,-701.0,-1001.0,-1301.0,93.4,65.8,-68.6,-374.0,-674.0,-974.0,-1274.0,-1007.0,-1307.0,-1607.0,-1907.0,-2207.0,-2507.0,-2807.0,-209.0,-509.0,-809.0,-1109.0,-1409.0,-1709.0,-2009.0,-1052.0,-1352.0,-1652.0,-1952.0,-2252.0,...,-788.0,-1088.0,-1388.0,-1688.0,-1988.0,166.1,166.1,166.1,352.4,-126.0,-326.0,-539.0,-301.0,-528.0,-797.0,-1097.0,-1397.0,-1697.0,-1997.0,-339.0,-571.0,-848.0,-1148.0,-1448.0,-1748.0,-2048.0,135.6,135.6,128.0,84.3,115.0,-248.0,-469.0,-292.0,-492.0,-788.0,-1088.0,-1388.0,-1688.0,-1988.0
4,78.9,13.7,-146.4,-367.0,-602.0,-878.0,-1178.0,-68.7,-245.5,-463.2,-724.0,-1015.0,-1313.0,-1613.0,-443.9,-702.0,-986.0,-1283.0,-1583.0,-1883.0,-2183.0,120.2,120.2,108.8,-1.9,-168.0,-404.0,-657.0,-185.2,-397.0,-645.0,-923.0,-1223.0,-1523.0,-1823.0,99.9,38.1,-117.0,-330.0,-564.0,...,-527.0,-827.0,-1127.0,-1427.0,-1727.0,-521.2,-821.0,-1121.0,-1421.0,-1721.0,-2021.0,-2321.0,-37.1,-262.9,-555.8,-857.0,-1157.0,-1457.0,-1757.0,107.6,104.9,20.7,-230.6,-536.0,-836.0,-1136.0,-1070.0,-1370.0,-1670.0,-1970.0,-2270.0,-2570.0,-2870.0,-31.5,-239.0,-545.0,-845.0,-1145.0,-1445.0,-1745.0
...,...,...,...,...,...,...,...,...,...,...,...,

In [18]:
# create columns for the column name of the max expected value, the max expected value, the contract having the max expected value.
def create_best_contracts(r):
    exp_tuples = tuple([(v,k) for k,v in r.items()])
    ex_tuples_sorted = sorted(exp_tuples,reverse=True)
    best_contract_tuple = ex_tuples_sorted[0]
    best_contract_split = best_contract_tuple[1].split('_')
    best_contract = best_contract_split[4]+best_contract_split[3]+best_contract_split[2]
    return [best_contract_tuple[1],best_contract_tuple[0],best_contract_tuple[0] if best_contract_tuple[1][-5] in ['N','S'] else -best_contract_tuple[0],best_contract]

sd_best_contract_l = sd_exp_df.apply(create_best_contracts,axis='columns')
sd_best_contract_df = pd.DataFrame(sd_best_contract_l.tolist(),columns=['Exp_Max_Col','Exp_Max','Exp_Max_NS','Best_Contract'])
sd_best_contract_df

,Exp_Max_Col,Exp_Max,Exp_Max_NS,Best_Contract
0,Exp_EW_W_S_1,124.4,-124.4,1SW
1,Exp_EW_W_S_1,124.4,-124.4,1SW
2,Exp_EW_W_S_4,352.4,-352.4,4SW
3,Exp_EW_W_S_4,352.4,-352.4,4SW
4,Exp_NS_S_C_2,121.2,121.2,2CS
...,...,...,...,...
315,Exp_EW_N_S_4,430.8,430.8,4SN
316,Exp_NS_N_N_3,511.8,511.8,3NN
317,Exp_NS_N_N_3,511.8,511.8,3NN
318,Exp_EW_S_N_3,342.6,342.6,3NS


In [19]:
merged_df = pd.concat([df,par_df,dd_tricks_df,dd_score_df,sd_best_contract_df],axis='columns')
merged_df

,deal,auction,play,board_num,_dealer,_vul,_contract,claimed,info,Event,Site,Date,West,North,East,South,Scoring,BCFlags,Room,Score,Par_NS,Par_EW,Par_Contracts_Result,DD_Tricks_N_C,DD_Tricks_N_D,DD_Tricks_N_H,DD_Tricks_N_S,DD_Tricks_N_N,DD_Tricks_S_C,DD_Tricks_S_D,DD_Tricks_S_H,DD_Tricks_S_S,DD_Tricks_S_N,DD_Tricks_E_C,DD_Tricks_E_D,DD_Tricks_E_H,DD_Tricks_E_S,DD_Tricks_E_N,DD_Tricks_W_C,DD_Tricks_W_D,...,DD_Score_7N_E,DD_Score_1C_W,DD_Score_2C_W,DD_Score_3C_W,DD_Score_4C_W,DD_Score_5C_W,DD_Score_6C_W,DD_Score_7C_W,DD_Score_1D_W,DD_Score_2D_W,DD_Score_3D_W,DD_Score_4D_W,DD_Score_5D_W,DD_Score_6D_W,DD_Score_7D_W,DD_Score_1H_W,DD_Score_2H_W,DD_Score_3H_W,DD_Score_4H_W,DD_Score_5H_W,DD_Score_6H_W,DD_Score_7H_W,DD_Score_1S_W,DD_Score_2S_W,DD_Score_3S_W,DD_Score_4S_W,DD_Score_5S_W,DD_Score_6S_W,DD_Score_7S_W,DD_Score_1N_W,DD_Score_2N_W,DD_Score_3N_W,DD_Score_4N_W,DD_Score_5N_W,DD_Score_6N_W,DD_Score_7N_W,Exp_Max_Col,Exp_Max,Exp_Max_NS,Best_Contract
0,N:T5.982.874.AQ632 K43.73.KQ5.KJT54 AJ9.AQT6.J...,"[P, 1♣, X, 1♠, P, 1NT, P, 2♥, P, 2♠, P, P, P]","[♦8, ♦5, ♦T, ♦A, ♣7, ♣A, ♣4, ♣8, ♠5, ♠3, ♠9, ♠...",1,0,0,2♠W+1,False,{'Event': '<u>Camrose 2024: BEN vs WBridge5</u...,<u>Camrose 2024: BEN vs WBridge5</u>,,2023.12.15,WBridge5,BENCAM22,WBridge5,BENCAM22,IMP,df,Open,EW 140,-140,140,"[1SE 2, 1SW 2]",5,5,5,4,5,5,6,6,4,5,8,7,7,9,8,8,7,...,-1100,90,90,-100,-300,-500,-800,-1100,70,-100,-300,-500,-800,-1100,-1400,80,-100,-300,-500,-800,-1100,-1400,140,140,140,-100,-300,-500,-800,120,120,-100,-300,-500,-800,-1100,Exp_EW_W_S_1,124.4,-124.4,1SW
1,N:T5.982.874.AQ632 K43.73.KQ5.KJT54 AJ9.AQT6.J...,"[P, 1♣, X, XX, P, P, 1♥, 1♠, P, 2♣, P, P, 2♥, ...","[♣7, ♣A, ♣5, ♣8, ♥2, ♥7, ♥Q, ♥K, ♠2, ♠T, ♠K, ♠...",1,0,0,2♥S-2,False,"{'Event': '', 'Site': '', 'Date': '2023.12.15'...",,,2023.12.15,BENCAM22,WBridge5,BENCAM22,WBridge5,IMP,97,Closed,NS -100,-140,140,"[1SE 2, 1SW 2]",5,5,5,4,5,5,6,6,4,5,8,7,7,9,8,8,7,...,-1100,90,90,-100,-300,-500,-800,-1100,70,-100,-300,-500,-800,-1100,-1400,80,-100,-300,-500,-800,-1100,-1400,140,140,140,-100,-300,-500,-800,120,120,-100,-300,-500,-800,-1100,Exp_EW_W_S_1,124.4,-124.4,1SW
2,N:T4.K62.KQ985.T54 J2.T9875.J4.AQ82 A73.AQJ43....,"[P, 1♥, 1♠, 2♥, P, P, 2♠, P, 3♠, P, P, P]","[♥6, ♥5, ♥A, ♠6, ♠8, ♠4, ♠J, ♠A, ♠3, ♠K, ♠T, ♠...",2,1,2,3♠W+1,False,"{'Event': '', 'Site': '', 'Date': '2023.12.15'...",,,2023.12.15,WBridge5,BENCAM22,WBridge5,BENCAM22,IMP,df,Open,EW 170,-420,420,"[4SE 0, 4SW 0]",2,7,7,3,5,2,7,7,3,5,11,5,6,10,5,11,5,...,-2000,150,150,150,150,400,-100,-300,-300,-500,-800,-1100,-1400,-1700,-2000,-100,-300,-500,-800,-1100,-1400,-1700,170,170,170,420,-100,-300,-500,-300,-500,-800,-1100,-1400,-1700,-2000,Exp_EW_W_S_4,352.4,-352.4,4SW
3,N:T4.K62.KQ985.T54 J2.T9875.J4.AQ82 A73.AQJ43....,"[P, P, 1♠, P, 1NT, 2♥, 2♠, 3♥, 3♠, P, 4♠, P, P...","[♥2, ♥7, ♥A, ♠6, ♠8, ♠4, ♠J, ♠3, ♦4, ♦2, ♦7, ♦...",2,1,2,4♠W+1,False,"{'Event': '', 'Site': '', 'Date': '2023.12.15'...",,,2023.12.15,BENCAM22,WBridge5,BENCAM22,WBridge5,IMP,97,Closed,EW 450,-420,420,"[4SE 0, 4SW 0]",2,7,7,3,5,2,7,7,3,5,11,5,6,10,5,11,5,...,-2000,150,150,150,150,400,-100,-300,-300,-500,-800,-1100,-1400,-1700,-2000,-100,-300,-500,-800,-1100,-1400,-1700,170,170,170,420,-100,-300,-500,-300,-500,-800,-1100,-1400,-1700,-2000,Exp_EW_W_S_4,352.4,-352.4,4SW
4,N:JT6.AK.972.T9754 K954.T3.QJ654.A6 AQ32.Q986....,"[1♣, 1♥, 2♣, P, 3♣, P, P, P]","[♦A, ♦2, ♦4, ♦3, ♦K, ♦7, ♦5, ♣3, ♥6, ♥2, ♥K, ♥...",3,2,3,3♣S+2,False,"{'Event': '', 'Site': '', 'Date': '2023.12.15'...",,,2023.12.15,WBridge5,BENCAM22,WBridge5,BENCAM22,IMP,df,Open,NS 150,400,-400,"[5CN 0, 5CS 0]",11,5,7,8,7,11,5,7,8,7,2,8,6,4,6,2,8,...,-1700,-1100,-1400,-1700,-2000,-2300,-2600,-2900,90,90,-100,-300,-500,-800,-1100,-300,-500,-800,-1100,-1400,-1700,-2000,-500,-800,-1100,-1400,-1700,-2000,-2300,-100,-300,-500,-800,-1100,-1400,-1700,Exp_NS_S_C_2,121.2,121.2,2CS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [20]:
def convert_contract_to_contract(r):
    return str(r['_contract']).upper().replace('♠','S').replace('♥','H').replace('♦','D').replace('♣','C').replace('NT','N')

def convert_contract_to_declarer(r):
    declarer = pd.NA if r['Contract'] == 'PASS' else r['Contract'][2]
    assert declarer is pd.NA or declarer in 'NSEW', f"declarer:{declarer}"
    return declarer

def declarer_to_declarer_name(r):
    declarer_name = pd.NA if r['Declarer'] is pd.NA else r[r['Declarer']]
    return declarer_name

def convert_contract_to_result(r):
    result = pd.NA if r['Contract'] == 'PASS' else 0 if r['Contract'][-1] in ['=','0'] else int(r['Contract'][-1]) if r['Contract'][-2] == '+' else -int(r['Contract'][-1])
    assert result is pd.NA or result in range(-13,8), f"result:{result}"
    return result

def convert_contract_to_tricks(r):
    tricks = pd.NA if r['Contract'] == 'PASS' else int(r['Contract'][0])+6+r['Result']
    assert tricks is pd.NA or tricks in range(0,14), f"tricks:{tricks}"
    return tricks

def convert_contract_to_dd_tricks(r):
    dd_tricks = 0 if r['Contract'] == 'PASS' else dd_tricks_df['_'.join(['DD_Tricks',r['Declarer'],r['Contract'][1]])].iloc[r.name]
    assert dd_tricks in range(0,14), f"dd_tricks:{dd_tricks}"
    return dd_tricks

def convert_score_to_score(r):
    score_split = r['_score'].split()
    assert len(score_split) == 2, f"score_split:{score_split}"
    assert score_split[0] in ['NS','EW'], f"score_split:{score_split[0]}"
    assert score_split[1][0] == '-' or str.isdigit(score_split[1][0]), f"score_split:{score_split[1]}"
    score_split_direction = score_split[0]
    score_split_value = score_split[1]
    score_value = -int(score_split_value) if score_split_value[0] == '-' else int(score_split_value)
    return score_value if score_split_direction == 'NS' else -score_value

cols = ['board_num','deal','Room','_contract','Score','_vul','Par_NS','Exp_Max_Col','Exp_Max','Exp_Max_NS','Best_Contract','North','East','South','West']
augmented_df = merged_df[cols].copy()
augmented_df.rename(columns={'North':'N','East':'E','South':'S','West':'W'},inplace=True)
augmented_df['Contract'] = augmented_df.apply(convert_contract_to_contract,axis='columns').astype('string')
augmented_df['Declarer'] = augmented_df.apply(convert_contract_to_declarer,axis='columns').astype('string')
augmented_df['Declarer_Name'] = augmented_df.apply(declarer_to_declarer_name,axis='columns').astype('string')
augmented_df['Result'] = augmented_df.apply(convert_contract_to_result,axis='columns').astype('Int16')
augmented_df['Tricks'] = augmented_df.apply(convert_contract_to_tricks,axis='columns').astype('UInt8')
augmented_df['DD_Tricks'] = augmented_df.apply(convert_contract_to_dd_tricks,axis='columns').astype('UInt8')
augmented_df.rename(columns={'Score':'_score'},inplace=True)
augmented_df['Score_NS'] = augmented_df.apply(convert_score_to_score,axis='columns').astype('int16')
augmented_df['Par_Diff_NS'] = augmented_df['Score_NS']-augmented_df['Par_NS'].astype('int16')
augmented_df['DD_Tricks_Diff'] = augmented_df['Tricks']-augmented_df['DD_Tricks'].astype('int8')
augmented_df['Exp_Max_Diff_NS'] = augmented_df['Score_NS']-augmented_df['Exp_Max_NS'].astype('int16')
augmented_df.drop(columns=['_contract','_score'],inplace=True)
augmented_df

,board_num,deal,Room,_vul,Par_NS,Exp_Max_Col,Exp_Max,Exp_Max_NS,Best_Contract,N,E,S,W,Contract,Declarer,Declarer_Name,Result,Tricks,DD_Tricks,Score_NS,Par_Diff_NS,DD_Tricks_Diff,Exp_Max_Diff_NS
0,1,N:T5.982.874.AQ632 K43.73.KQ5.KJT54 AJ9.AQT6.J...,Open,0,-140,Exp_EW_W_S_1,124.4,-124.4,1SW,BENCAM22,WBridge5,BENCAM22,WBridge5,2SW+1,W,WBridge5,1,9,9,-140,0,0,-16
1,1,N:T5.982.874.AQ632 K43.73.KQ5.KJT54 AJ9.AQT6.J...,Closed,0,-140,Exp_EW_W_S_1,124.4,-124.4,1SW,WBridge5,BENCAM22,WBridge5,BENCAM22,2HS-2,S,WBridge5,-2,6,6,100,240,0,224
2,2,N:T4.K62.KQ985.T54 J2.T9875.J4.AQ82 A73.AQJ43....,Open,2,-420,Exp_EW_W_S_4,352.4,-352.4,4SW,BENCAM22,WBridge5,BENCAM22,WBridge5,3SW+1,W,WBridge5,1,10,10,-170,250,0,182
3,2,N:T4.K62.KQ985.T54 J2.T9875.J4.AQ82 A73.AQJ43....,Closed,2,-420,Exp_EW_W_S_4,352.4,-352.4,4SW,WBridge5,BENCAM22,WBridge5,BENCAM22,4SW+1,W,BENCAM22,1,11,10,-450,-30,1,-98
4,3,N:JT6.AK.972.T9754 K954.T3.QJ654.A6 AQ32.Q986....,Open,3,400,Exp_NS_S_C_2,121.2,121.2,2CS,BENCAM22,WBridge5,BENCAM22,WBridge5,3CS+2,S,BENCAM22,2,11,11,150,-250,0,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,158,N:T732.KJT9.K6.KT7 A.74.AQJ972.AJ42 KQJ985.AQ8...,Closed,0,450,Exp_EW_N_S_4,430.8,430.8,4SN,WBridge5,BENCAM22,WBridge5,BENCAM22,5DEX-2,E,BENCAM22,-2,9,8,-300,-750,1,-730
316,159,N:QT2.QJ9.AKJ.AJ65 K84.AT85.842.Q97 A95.6432.T...,Open,2,630,Exp_NS_N_N_3,511.8,511.8,3NN,BENCAM22,WBridge5,BENCAM22,WBridge5,3NN-3,N,BENCAM22,-3,6,10,300,-330,-4,-211
317,159,N:QT2.QJ9.AKJ.AJ65 K84.AT85.842.Q97 A95.6432.T...,Closed,2,630,Exp_NS_N_N_3,511.8,511.8,3NN,WBridge5,BENCAM22,WBridge5,BENCAM22,1NN+3,N,WBridge5,3,10,10,180,-450,0,-331
318,160,N:843.9765.A73.AK4 T65.KQ82.Q52.T93 AK.AJT.986...,Open,3,430,Exp_EW_S_N_3,342.6,342.6,3NS,BENCAM22,WBridge5,BENCAM22,WBridge5,2NS+2,S,BENCAM22,2,10,10,180,-250,0,-162


Following cells contain WIP experiments with comparative statistics; BENCAM22 vs WBridge5, Open vs Closed rooms, Tricks vs DD, par diffs, expected max diffs.

In [21]:
# describe() over Par_Diff_NS for all, bencam22, wbridge5
print('Describe North, BENCAM22, Par_Diff_NS:')
print(augmented_df[augmented_df['N'].eq('BENCAM22')]['Par_Diff_NS'].describe())
print('Describe North, WBridge5, Par_Diff_NS:')
print(augmented_df[augmented_df['N'].eq('WBridge5')]['Par_Diff_NS'].describe())

# sum over Par_Diff_NS for all, bencam22, wbridge5
all, bencam22, wbridge5 = augmented_df['Par_Diff_NS'].sum(),augmented_df[augmented_df['N'].eq('BENCAM22')]['Par_Diff_NS'].sum(),augmented_df[augmented_df['N'].eq('WBridge5')]['Par_Diff_NS'].sum()
print(f"Sum of Par_Diff_NS: All:{all} BENCAM22:{bencam22} WBridge5:{wbridge5} BENCAM22-WBridge5:{bencam22-wbridge5}")

# frequency where par was exceeded for all, bencam22, wbridge5
all, bencam22, wbridge5 = sum(augmented_df['Par_Diff_NS'].gt(0)),sum(augmented_df['N'].eq('BENCAM22')&augmented_df['Par_Diff_NS'].gt(0)),sum(augmented_df['N'].eq('WBridge5')&augmented_df['Par_Diff_NS'].gt(0))
print(f"Frequency where exceeding Par: All:{all} BENCAM22:{bencam22} WBridge5:{wbridge5} BENCAM22-WBridge5:{bencam22-wbridge5}")

Describe North, BENCAM22, Par_Diff_NS:
count     160.00000
mean      -29.62500
std       487.34667
min     -2240.00000
25%      -252.50000
50%        -5.00000
75%        60.00000
max      1700.00000
Name: Par_Diff_NS, dtype: float64
Describe North, WBridge5, Par_Diff_NS:
count     160.000000
mean       34.500000
std       382.222564
min     -1550.000000
25%       -82.500000
50%         0.000000
75%       240.000000
max      1230.000000
Name: Par_Diff_NS, dtype: float64
Sum of Par_Diff_NS: All:780 BENCAM22:-4740 WBridge5:5520 BENCAM22-WBridge5:-10260
Frequency where exceeding Par: All:135 BENCAM22:60 WBridge5:75 BENCAM22-WBridge5:-15


In [22]:
# describe() over DD_Tricks_Diff for all, bencam22, wbridge5
print('Describe Declarer, BENCAM22, DD_Tricks_Diff:')
print(augmented_df[augmented_df['Declarer_Name'].eq('BENCAM22')]['DD_Tricks_Diff'].describe())
print('Describe Declarer, WBridge5, DD_Tricks_Diff:')
print(augmented_df[augmented_df['Declarer_Name'].eq('WBridge5')]['DD_Tricks_Diff'].describe())

# sum over DD_Tricks_Diff for all, bencam22, wbridge5
all, bencam22, wbridge5 = augmented_df['DD_Tricks_Diff'].sum(),augmented_df[augmented_df['Declarer_Name'].eq('BENCAM22')]['DD_Tricks_Diff'].sum(),augmented_df[augmented_df['Declarer_Name'].eq('WBridge5')]['DD_Tricks_Diff'].sum()
print(f"Sum of DD_Tricks_Diff: All:{all} BENCAM22:{bencam22} WBridge5:{wbridge5} BENCAM22-WBridge5:{bencam22-wbridge5}")

# frequency where Tricks > DD for all, bencam22, wbridge5
all, bencam22, wbridge5 = sum(augmented_df['DD_Tricks_Diff'].notna() & augmented_df['DD_Tricks_Diff'].gt(0)),sum(augmented_df[augmented_df['Declarer_Name'].eq('BENCAM22')]['DD_Tricks_Diff'].gt(0)),sum(augmented_df[augmented_df['Declarer_Name'].eq('WBridge5')]['DD_Tricks_Diff'].gt(0))
print(f"Frequency where Tricks > DD: All:{all} BENCAM22:{bencam22} WBridge5:{wbridge5} BENCAM22-WBridge5:{bencam22-wbridge5}")

# frequency where Tricks < DD for all, bencam22, wbridge5
all, bencam22, wbridge5 = sum(augmented_df['DD_Tricks_Diff'].notna() & augmented_df['DD_Tricks_Diff'].lt(0)),sum(augmented_df[augmented_df['Declarer_Name'].eq('BENCAM22')]['DD_Tricks_Diff'].lt(0)),sum(augmented_df[augmented_df['Declarer_Name'].eq('WBridge5')]['DD_Tricks_Diff'].lt(0))
print(f"Frequency where Tricks < DD: All:{all} BENCAM22:{bencam22} WBridge5:{wbridge5} BENCAM22-WBridge5:{bencam22-wbridge5}")

Describe Declarer, BENCAM22, DD_Tricks_Diff:
count       147.0
mean    -0.020408
std       1.00321
min          -4.0
25%           0.0
50%           0.0
75%           1.0
max           2.0
Name: DD_Tricks_Diff, dtype: Float64
Describe Declarer, WBridge5, DD_Tricks_Diff:
count       168.0
mean     0.095238
std      0.961773
min          -3.0
25%           0.0
50%           0.0
75%           1.0
max           3.0
Name: DD_Tricks_Diff, dtype: Float64
Sum of DD_Tricks_Diff: All:13 BENCAM22:-3 WBridge5:16 BENCAM22-WBridge5:-19
Frequency where Tricks > DD: All:85 BENCAM22:38 WBridge5:47 BENCAM22-WBridge5:-9
Frequency where Tricks < DD: All:67 BENCAM22:33 WBridge5:34 BENCAM22-WBridge5:-1


In [23]:
# describe() over Par_Diff_NS for all, open, closed
print(augmented_df['Par_Diff_NS'].describe(),augmented_df[augmented_df['Room'].eq('Open')]['Par_Diff_NS'].describe(),augmented_df[augmented_df['Room'].eq('Closed')]['Par_Diff_NS'].describe())
# sum over Par_Diff_NS for all, bencam22, wbridge5
all, bencam22, wbridge5 = augmented_df['Par_Diff_NS'].sum(),augmented_df[augmented_df['Room'].eq('Open')]['Par_Diff_NS'].sum(),augmented_df[augmented_df['Room'].eq('Closed')]['Par_Diff_NS'].sum()
print(f"Sum of Par_Diff_NS: All:{all} BENCAM22:{bencam22} WBridge5:{wbridge5} BENCAM22-WBridge5:{bencam22-wbridge5}")
all, open, closed = sum(augmented_df['Par_Diff_NS'].gt(0)),sum(augmented_df['Room'].eq('Open')&augmented_df['Par_Diff_NS'].gt(0)),sum(augmented_df['Room'].eq('Closed')&augmented_df['Par_Diff_NS'].gt(0))
print(f"Frequency where exceeding Par: All:{all} Open:{open} Closed:{closed} Open-Closed:{open-closed}")

count     320.000000
mean        2.437500
std       438.440878
min     -2240.000000
25%      -220.000000
50%         0.000000
75%       190.000000
max      1700.000000
Name: Par_Diff_NS, dtype: float64 count     160.00000
mean      -29.62500
std       487.34667
min     -2240.00000
25%      -252.50000
50%        -5.00000
75%        60.00000
max      1700.00000
Name: Par_Diff_NS, dtype: float64 count     160.000000
mean       34.500000
std       382.222564
min     -1550.000000
25%       -82.500000
50%         0.000000
75%       240.000000
max      1230.000000
Name: Par_Diff_NS, dtype: float64
Sum of Par_Diff_NS: All:780 BENCAM22:-4740 WBridge5:5520 BENCAM22-WBridge5:-10260
Frequency where exceeding Par: All:135 Open:60 Closed:75 Open-Closed:-15


In [24]:
# describe() over Exp_Max_Diff_NS for all, open, closed
print(augmented_df['Exp_Max_Diff_NS'].describe(),augmented_df[augmented_df['Room'].eq('Open')]['Exp_Max_Diff_NS'].describe(),augmented_df[augmented_df['Room'].eq('Closed')]['Exp_Max_Diff_NS'].describe())
# sum over Exp_Max_Diff_NS for all, bencam22, wbridge5
all, bencam22, wbridge5 = augmented_df['Exp_Max_Diff_NS'].sum(),augmented_df[augmented_df['Room'].eq('Open')]['Exp_Max_Diff_NS'].sum(),augmented_df[augmented_df['Room'].eq('Closed')]['Exp_Max_Diff_NS'].sum()
print(f"Sum of Exp_Max_Diff_NS: All:{all} BENCAM22:{bencam22} WBridge5:{wbridge5} BENCAM22-WBridge5:{bencam22-wbridge5}")
all, open, closed = sum(augmented_df['Exp_Max_Diff_NS'].gt(0)),sum(augmented_df['Room'].eq('Open')&augmented_df['Exp_Max_Diff_NS'].gt(0)),sum(augmented_df['Room'].eq('Closed')&augmented_df['Exp_Max_Diff_NS'].gt(0))
print(f"Frequency where exceeding Exp_Max_Diff_NS: All:{all} Open:{open} Closed:{closed} Open-Closed:{open-closed}")

count     320.000000
mean      -32.425000
std       371.657198
min     -1855.000000
25%      -200.500000
50%         1.000000
75%       141.250000
max      1725.000000
Name: Exp_Max_Diff_NS, dtype: float64 count     160.000000
mean      -64.487500
std       414.714086
min     -1855.000000
25%      -245.000000
50%       -16.000000
75%       127.750000
max      1725.000000
Name: Exp_Max_Diff_NS, dtype: float64 count     160.000000
mean       -0.362500
std       321.045432
min     -1430.000000
25%      -118.750000
50%         9.000000
75%       146.000000
max      1231.000000
Name: Exp_Max_Diff_NS, dtype: float64
Sum of Exp_Max_Diff_NS: All:-10376 BENCAM22:-10318 WBridge5:-58 BENCAM22-WBridge5:-10260
Frequency where exceeding Exp_Max_Diff_NS: All:161 Open:73 Closed:88 Open-Closed:-15
